In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors import LocalOutlierFactor
df=pd.read_csv('a.csv')
x = df['V_in']
y = df['Vibration_1']
X1, X2, y1, y2 = train_test_split(x, y, test_size=0.00077)
test= pd.concat([X2,y2], axis=1)
test

In [ ]:
plt.scatter(X2, y2, marker='o', linestyle='-',s=1)
plt.xlabel('voltage')
plt.ylabel('vibration')
plt.grid(True)
plt.show()
df= pd.DataFrame()
df['v']=X2
df['vibration1']=y2
series=pd.Series(y2)

In [ ]:
mean_value = series.mean()
std_deviation=series.std()
model = LocalOutlierFactor(n_neighbors=40, contamination=std_deviation*.707+mean_value)
outlier_scores = model.fit_predict(df)
negative_outlier_scores = model.negative_outlier_factor_
print(outlier_scores)
df['anomoly']=outlier_scores
df['outlier_scores']=-negative_outlier_scores
plt.figure(figsize=(10, 6))
plt.scatter(df['v'], df['vibration1'], c=df['anomoly'], cmap='viridis', s=1)
plt.colorbar(label='LOF Score')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('LOF Density Plot')
plt.show()

In [ ]:
import math
def dist(a1,a2,b1,b2):
 return math.sqrt((a1 - a2) ** 2+(b1 - b2) ** 2)
vibe=np.array(df['vibration1'])
volt=np.array(df['v'])
segments = []
current_segment = [vibe[0]]
c_volt=[volt[0]]
thershold=0
j=1
for i in range(1, len(vibe)):
    distance = dist(vibe[i],current_segment[-1],volt[i],c_volt[-1])
    if (((distance <= (thershold/j)*5 ) | (j==1))&(j<100)):
        j=j+1
        current_segment.append(vibe[i])
        c_volt.append(volt[i])
        thershold=distance+thershold
    else:
        j=1
        segments.append(current_segment)
        current_segment = [vibe[i]]
        c_volt=[volt[i]]
        thershold=0
segments.append(current_segment)

In [ ]:
from scipy.stats import entropy
import numpy as np
csegments=pd.Series(segments)
Div_k = [abs(1 - entropy(pd.Series(segment).value_counts(normalize=True))) for segment in csegments]
Div_k
print(len(Div_k))
wei=[(100*Div_k[i]/np.sum(Div_k[0:len(Div_k)]))+0*y for i,segment in enumerate(segments) for y in segment]
print("Weighted Matrix:")
print(wei)
print(len(wei))
df['weight']=wei

In [ ]:
n_anomoly=df[df['anomoly']==1]
n_anomoly
s_anomoly=df[df['anomoly']==-1]
plt.figure(figsize=(10, 6))
plt.scatter(n_anomoly['v'], n_anomoly['vibration1'], c=n_anomoly['weight'], cmap='viridis', s=1)
plt.colorbar(label='LOF Score')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('LOF Density Plot')
plt.show()
plt.figure(figsize=(10, 6))
plt.scatter(s_anomoly['v'], s_anomoly['vibration1'], c=s_anomoly['weight'], cmap='viridis', s=1)
plt.colorbar(label='LOF Score')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('LOF Density Plot')
plt.show()

In [ ]:
uw=[]
for i,w in enumerate(df['weight']):
 uw.append(w*.5*-negative_outlier_scores[i])
wei=pd.Series(uw)
t=pd.Series.std(wei)+pd.Series.mean(wei)
updated_anomoly=[]
j=0
for i,w in enumerate(df['weight']):
    if (w>t):
        j=j+1
        updated_anomoly.append(outlier_scores[i]*-1)
    else:
        updated_anomoly.append(outlier_scores[i])
        #print("else")
df['ua']=updated_anomoly
plt.figure(figsize=(10, 6))
plt.scatter(df['v'], df['vibration1'], c=df['ua'], s=1)
plt.colorbar(label='LOF Score')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('LOF Density Plot')
plt.show()

In [ ]:
from sklearn.metrics import silhouette_score
clusters = np.where(outlier_scores == -1, 0, 1)
silhouette_avg = silhouette_score(df,clusters)
print("The average silhouette_score is :", silhouette_avg)

clusters = np.where(df['ua'] == -1,0, 1)
silhouette_avg = silhouette_score(df,clusters)
print("The average silhouette_score is :", silhouette_avg)